In [1]:
import zipfile
import io
from google.colab import files

# Assuming the uploaded file is named 'American-Sign-Language-Letters.zip'
zip_file_name = 'American-Sign-Language-Letters.zip'
zip_data = files.upload()[zip_file_name]

# Create a file-like object from the uploaded data
zip_buffer = io.BytesIO(zip_data)

# Extract the contents of the ZIP file
with zipfile.ZipFile(zip_buffer, 'r') as zip_ref:
    zip_ref.extractall('/content/American-Sign-Language-Letters/')


KeyError: 'American-Sign-Language-Letters.zip'

In [ ]:
import os
import shutil

# Define paths to 'test' and 'valid' directories
test_path = '/content/American-Sign-Language-Letters/test'
valid_path = '/content/American-Sign-Language-Letters/valid'

# Function to organize images into separate class folders
def organize_images_into_classes(folder_path):
    for img_name in os.listdir(folder_path):
        # Extract the first letter from the image name
        first_letter = img_name[0].lower()

        # Define the destination folder based on the first letter
        destination_folder = os.path.join(folder_path, first_letter)

        # Create the destination folder if it doesn't exist
        os.makedirs(destination_folder, exist_ok=True)

        # Move the image to its respective class folder
        src_path = os.path.join(folder_path, img_name)
        dst_path = os.path.join(destination_folder, img_name)
        shutil.move(src_path, dst_path)

# Organize images in the 'test' folder into separate class folders
organize_images_into_classes(test_path)

# Organize images in the 'valid' folder into separate class folders
organize_images_into_classes(valid_path)


In [ ]:
import os
import shutil

# Define path to 'train' directory
train_path = '/content/American-Sign-Language-Letters/train'

# Function to organize images into separate class folders
def organize_images_into_classes(folder_path):
    for img_name in os.listdir(folder_path):
        # Extract the first letter from the image name
        first_letter = img_name[0].lower()

        # Define the destination folder based on the first letter
        destination_folder = os.path.join(folder_path, first_letter)

        # Create the destination folder if it doesn't exist
        os.makedirs(destination_folder, exist_ok=True)

        # Move the image to its respective class folder
        src_path = os.path.join(folder_path, img_name)
        dst_path = os.path.join(destination_folder, img_name)
        shutil.move(src_path, dst_path)

# Organize images in the 'train' folder into separate class folders
organize_images_into_classes(train_path)


In [ ]:
import os

# Define the base folder path
base_folder_path = '/content/American-Sign-Language-Letters/test'  # Change this to the path of your 'test' folder

# List of class names (modify accordingly)
class_names = ['e','l']

# Create empty folders for each class
for class_name in class_names:
    class_folder_path = os.path.join(base_folder_path, class_name)
    os.makedirs(class_folder_path, exist_ok=True)
    print(f"Created folder: {class_folder_path}")


Created folder: /content/American-Sign-Language-Letters/test/e
Created folder: /content/American-Sign-Language-Letters/test/l


In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.8, patience=2, min_lr=1e-5)
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

In [ ]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input, decode_predictions
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

# Define the base folder path for your dataset
base_folder_path = '/content/American-Sign-Language-Letters/'

# Load the pre-trained VGG16 model without the top (fully connected) layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False

# Add your custom top layers for your specific task
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(26, activation='softmax')(x)  # Assuming 26 classes for sign language

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Display the model summary
model.summary()

# Set up data generators
# train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
# valid_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    base_folder_path + 'train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

valid_generator = valid_datagen.flow_from_directory(
    base_folder_path + 'valid',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

# Train the model
history=model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=50,
    validation_data=valid_generator,
    validation_steps=len(valid_generator),
    callbacks=[reduce_lr, early_stopping])


58889256/58889256 [==============================] - 0s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                              

In [ ]:
# Assuming 'model' is your trained model
model.save('utpalMC1.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
